# Purpose
Identify data gaps in the ownership data.

## Overview
Review linkages which are apparent in the ownership dataset for:
- Main Line Health
- Jefferson Health
- Penn Medicine

## Key Questions
### Are the `associate_id` drawing a link to effectively characterize the "health system"?



In [1]:
import os
os.chdir("..")
os.chdir("..")
os.getcwd()

'/home/dr00b/code/slothful_sleuth'

In [2]:
%load_ext sql

In [30]:
%sql sqlite:///data/raw/cms_data_api_raw.db
%config SqlMagic.displaylimit = 100

In [33]:

%%sql
SELECT COUNT(1) FROM hospital_enrollments e

-- SELECT * FROM hospital_enrollments e
-- WHERE e.`STATE` = 'PA'
-- ORDER BY e.`ORGANIZATION NAME` ASC

--AND e.`ORGANIZATION NAME` = 'MAIN LINE HOSPITALS, INC.'

-- INNER JOIN hospital_all_owners o
-- ON e.`ENROLLMENT ID`  = o.`ENROLLMENT ID`
-- AND e.`STATE` = 'PA'
--AND e.STATE = 'PA'-- AND e.`ORGANIZATION NAME` LIKE '%MAIN LINE%' 


Running query in 'sqlite:///data/raw/cms_data_api_raw.db'

COUNT(1)
1000
